In [ ]:
# wandb and pytorch manual 
# https://colab.research.google.com/drive/1XDtq-KT0GkX06a_g1MevuLFOMk4TxjKZ#scrollTo=bZpt5W2NNl6S

# for hyperparameter sweeping
# https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=W_YSZCkITmVJ


In [ ]:
from __future__ import print_function
import os
import argparse
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torchvision import datasets, transforms

import logging
logging.propagate = False
logging.getLogger().setLevel(logging.ERROR)

import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime
import wandb

from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

In [ ]:
!wandb login

## hyperparameter sweep using wandb

In [ ]:
# use random search
sweep_config = {
    'method': 'random'
    }

# for bayesian search, this value should be included
metric = {
    'name': 'f1_score',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric    

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        # 'values': ['adam']
        },
    # 'fc_layer_size': {
    #     'values': [128, 256, 512]
    #     },
    # 'dropout': {
    #       'values': [0.3, 0.4, 0.5]
    #     },
    }

parameters_dict.update({
    'epochs': {
        'value': 100}
    })

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'lr': {
        # a q_log_uniform_values distribution between 0 and 0.1
        'distribution': 'q_log_uniform_values',
        'q': 1e-5,
        'min': 1e-4,
        'max': 5e-2
      },
    'batch_size': {
        'values': [128,256,512,1024]
      },
    'svd_reduction': {
        'values': [True, False]
      },    
    # 'momentum': {
    #     # float between 0.1 and 0.9
    #     # with evenly-distributed logarithms 
    #     'distribution': 'uniform',
    #     'min': 0.1,
    #     'max': 0.9,
    #   },
    # 'weight_decay': {
    #     # float between 1e-3 and 1e-5
    #     # with evenly-distributed logarithms 
    #     'distribution': 'q_log_uniform_values',
    #     'q': 10,
    #     'min': 1e-5,
    #     'max': 1e-3,
    #   },    
    })

In [ ]:
import pprint

pprint.pprint(sweep_config)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Credit Card Fraud Detection Sweep")

In [ ]:

# now = datetime.now().strftime('%Y%m%d_%H%M%S')
# wandb.init(project='Credit Card Fraud Detection',  name=now, mode='online')
# wandb.watch_called = False

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# config =wandb.config
# config.batch_size = 1024
# config.epochs = 100
# config.lr = 1e-2
# config.momentum = 0.5
# config.weight_decay = 1e-4
# config.device = device
# config.seed = 42
# config.log_interval = 10
# config.num_workers = 8
# config.adam = False


# set seed for reproduce

In [ ]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


## For SVD reduction

In [ ]:
def reduced_mat(df) :
    u, s, v = np.linalg.svd(df, full_matrices=False)
    s[0] = 0
    rdf = pd.DataFrame( u @ np.diag(s) @ v)
    return rdf

# dataset

In [ ]:
# Pandas is 18 times slower than Numpy (15.8ms vs 0.874 ms). Pandas is 20 times slower than Numpy
# therefore Use numpy for faster data loading

class CDataset(Dataset):
    def __init__(self, df, eval_mode=False):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [ ]:
# train_dataset = CDataset(train_df)
# train_loader = DataLoader(train_dataset, batch_size = config.batch_size, shuffle=True, num_workers=config.num_workers)

# val_dataset = CDataset(val_df, eval_mode=True)
# val_loader = DataLoader(val_dataset, batch_size = config.batch_size, shuffle=False, num_workers=config.num_workers)

## model

In [ ]:
class AutoEncoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        # self.act = nn.ReLU()
        self.act = nn.GELU()
        # self.act = nn.LeakyReLU()
        
        self.dim = 30
        self.hidden1 = 64
        self.hidden2 = 128
        
        self.encoder = nn.Sequential(
            nn.BatchNorm1d(self.dim),
            nn.Linear(self.dim,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.hidden2),
            nn.BatchNorm1d(self.hidden2),
            self.act,
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.hidden2,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.dim),
        )
        
#         self.encoder2 = nn.Sequential(
#             nn.BatchNorm1d(self.dim),
#             nn.Linear(self.dim,self.hidden1),
#             nn.BatchNorm1d(self.hidden1),
#             self.act,
#             nn.Linear(self.hidden1,self.hidden2),
#             nn.BatchNorm1d(self.hidden2),
#             self.act,
#         )
        
#         self.decoder2 = nn.Sequential(
#             nn.Linear(self.hidden2,self.hidden1),
#             nn.BatchNorm1d(self.hidden1),
#             self.act,
#             nn.Linear(self.hidden1,self.dim),
#         )        
        
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)          
        
    def forward(self, x) :
        
        x = self.encoder(x)
        x = self.decoder(x)
        
#         x = self.encoder2(x1)
#         x = self.decoder2(x)
        
        return x
        

In [ ]:
class Trainer() :
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, config) :
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = config.device
        self.epochs = config.epochs
        self.lr = config.lr
        
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self,) :
        self.model.to(self.device)
        best_score = 0
        for epoch in range(self.epochs) :
            self.model.train()
            train_loss = []
            
            for x in iter(self.train_loader) :
                x = x.to(self.device)
                _x = self.model(x)
                
                loss = self.criterion(x, _x)
                # loss = self.criterion(x, _x) + 0.5*self.criterion(x, x1)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
                wandb.log({'train loss' : loss.item()})
                
            score = self.validation(self.model, 0.95)
            wandb.log({'f1_score' : score})
            
            if self.scheduler is not None :
                self.scheduler.step(score)
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}]')
            
            # for param_group in self.optimizer.param_groups:
            #     print(param_group['lr'])      
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}] lr [{self.scheduler.get_lr()}]')

            if best_score < score :
                best_score = score
                
            wandb.log({'best_score' : best_score})
#                 torch.save(self.model.state_dict(), '../saved/best_model.pth')
            
    def validation(self, eval_model, threshold) :
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred_y = []
        true_y = []
        
        with torch.no_grad():
            for x, y in iter(self.val_loader) :
                x = x.to(self.device)
                y = y.to(self.device)
                
                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff) < threshold, 1, 0).tolist()
                pred_y += batch_pred
                true_y += y.tolist()

        return f1_score(true_y, pred_y, average='macro')

In [ ]:

def main() :
    
    with wandb.init(config=None) :
        
        config = wandb.config
        config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        config.num_workers = 8
        config.seed = 42
        config.momentum = 0.9
        config.weight_decay = 1e-4

        train_df = pd.read_csv('../dataset/train.csv')
        val_df = pd.read_csv('../dataset/val.csv')
        train_df = train_df.drop(columns=['ID'])
        val_df = val_df.drop(columns=['ID'])  
        
        if config.svd_reduction :
            train_df = reduced_mat(train_df)
            val = reduced_mat(val_df.drop(columns=['Class']))
            val['Class'] = val_df['Class']
            val_df = val

        train_dataset = CDataset(train_df)
        train_loader = DataLoader(train_dataset, batch_size = config.batch_size, shuffle=True, num_workers=config.num_workers)

        val_dataset = CDataset(val_df, eval_mode=True)
        val_loader = DataLoader(val_dataset, batch_size = config.batch_size, shuffle=False, num_workers=config.num_workers)    

        seed_everything(config.seed)    

        model = AutoEncoder()
        model.eval()

        # if config.adam :
        if config.optimizer == 'adam' :
            optimizer = torch.optim.Adam(params=model.parameters(), lr = config.lr)
        else :
            optimizer = torch.optim.SGD(model.parameters(), config.lr,
                                        momentum=config.momentum,
                                        weight_decay=config.weight_decay)

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
        # scheduler = StepLR(optimizer, step_size=50, gamma=0.2)

        wandb.watch(model, log='all')

        trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, config)

        wandb.save('model.h5')

        trainer.fit()

In [ ]:
# if __name__ == '__main__':
#     main(config)

In [ ]:
wandb.agent(sweep_id, main, count=20)